In [1]:
import pandas as pd

# Load the dataset with only the necessary columns
selected_columns = ['Severity', 'Description', 'Street', 'City', 'State', 
                    'Country', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)',
                    'Pressure(in)', 'Visibility(mi)', 'Wind_Direction', 
                    'Wind_Speed(mph)', 'Weather_Condition']

df = pd.read_csv('E:/Master_thesis_REV/Datasets/Dataset_US_accidents/archive/US_Accidents_March23.csv', usecols=selected_columns)

# Display the first few rows to check the data
print(df.head())


   Severity                                        Description  \
0         3  Right lane blocked due to accident on I-70 Eas...   
1         2  Accident on Brice Rd at Tussing Rd. Expect del...   
2         2  Accident on OH-32 State Route 32 Westbound at ...   
3         3  Accident on I-75 Southbound at Exits 52 52B US...   
4         2  Accident on McEwen Rd at OH-725 Miamisburg Cen...   

                      Street          City State Country  Temperature(F)  \
0                     I-70 E        Dayton    OH      US            36.9   
1                   Brice Rd  Reynoldsburg    OH      US            37.9   
2             State Route 32  Williamsburg    OH      US            36.0   
3                     I-75 S        Dayton    OH      US            35.1   
4  Miamisburg Centerville Rd        Dayton    OH      US            36.0   

   Wind_Chill(F)  Humidity(%)  Pressure(in)  Visibility(mi) Wind_Direction  \
0            NaN         91.0         29.68            10.0         

In [2]:
# Handle missing values - example with 'Wind_Speed(mph)'
df['Wind_Speed(mph)'].fillna(df['Wind_Speed(mph)'].mean(), inplace=True)

# Frequency encoding for categorical columns
categorical_columns = ['Description', 'Street', 'City', 'State', 'Country', 'Weather_Condition', 'Wind_Direction']

for col in categorical_columns:
    freq = df[col].value_counts()
    df[col] = df[col].map(freq)

# Drop any remaining rows with missing values if necessary
df = df.dropna()

# Display the first few rows after preprocessing
print(df.head())


   Severity  Description   Street     City   State  Country  Temperature(F)  \
2         2          1.0    252.0   3010.0  118115  7728394            36.0   
3         3          5.0  27546.0  24572.0  118115  7728394            35.1   
4         2          2.0    729.0  24572.0  118115  7728394            36.0   
5         3          1.0    104.0   1073.0  118115  7728394            37.9   
6         2          1.0     26.0  24572.0  118115  7728394            34.0   

   Wind_Chill(F)  Humidity(%)  Pressure(in)  Visibility(mi)  Wind_Direction  \
2           33.3        100.0         29.67            10.0        364470.0   
3           31.0         96.0         29.64             9.0        364470.0   
4           33.3         89.0         29.65             6.0        364470.0   
5           35.5         97.0         29.63             7.0        384840.0   
6           31.0        100.0         29.66             7.0        353806.0   

   Wind_Speed(mph)  Weather_Condition  
2         

In [3]:
# Define features (X) and target (y)
X = df.drop(columns=['Severity'])
y = df['Severity']

# Display the shapes of X and y to confirm
print(f'X shape: {X.shape}, y shape: {y.shape}')


X shape: (5676313, 13), y shape: (5676313,)


In [4]:
from sklearn.model_selection import train_test_split

# Function to sample data
def sample_data(X, y, sample_size):
    if isinstance(sample_size, float):
        if 0 < sample_size < 1.0:
            return train_test_split(X, y, test_size=0.2, train_size=sample_size, random_state=42)
        elif sample_size == 1.0:
            return train_test_split(X, y, test_size=0.001, random_state=42)
        else:
            raise ValueError("sample_size as float must be in the range (0.0, 1.0) or equal to 1.0.")
    elif isinstance(sample_size, int):
        if sample_size > len(X):
            raise ValueError(f"sample_size {sample_size} exceeds the number of available samples {len(X)}.")
        sampled_X = X.sample(n=sample_size, random_state=42)
        sampled_y = y.loc[sampled_X.index]
        return train_test_split(sampled_X, sampled_y, test_size=0.2, random_state=42)
    else:
        raise ValueError("sample_size must be a float or an integer")


In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import numpy as np
import time
import psutil
import pandas as pd


In [6]:
# Function to calculate and return metrics
def calculate_metrics(X_train, X_test, y_train, y_test):
    rf = RandomForestRegressor(random_state=42)

    # Define hyperparameters for RandomizedSearchCV
    param_distributions = {
        'n_estimators': [50],  # Number of trees
        'max_depth': [2,5],  # Depth of trees
        'min_samples_split': [2, 4],  # Min samples to split
        'min_samples_leaf': [2, 4, 10],  # Min samples in leaf
        'max_features': ['sqrt', 'log2'],  # Features to consider for best split
        'bootstrap': [True]  # Whether to bootstrap samples
    }

    random_search = RandomizedSearchCV(rf, param_distributions, n_iter=10, cv=3, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1)

    start_time = time.time()
    start_cpu = psutil.cpu_percent(interval=None)
    random_search.fit(X_train, y_train)
    end_cpu = psutil.cpu_percent(interval=None)
    end_time = time.time()

    # Calculate time and CPU usage
    execution_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    y_pred = random_search.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    # Calculate the range of the target variable
    target_range = y_train.max() - y_train.min()

    # Calculate normalized RMSE (nRMSE)
    nrmse = rmse / target_range
    
    memory_usage_MB = X_train.memory_usage(deep=True).sum() / (1024 ** 2)
    normalized_time = execution_time / memory_usage_MB
    
    return {
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2,
        'MSE': mse,
        'nRMSE': nrmse,  # Normalized RMSE
        'Execution Time (Raw)': execution_time,  # Raw execution time
        'Normalized Time (s/MB)': normalized_time,  # Normalized execution time
        'Average CPU Usage': avg_cpu_usage
    }


In [7]:
# Define sample sizes
sample_sizes = [1.0, 0.5, 0.25, 0.125, 100, 1000, 10000, 100000]

# Initialize metrics storage
metrics_list = []
total_execution_time = 0
total_cpu_usage = 0
total_memory_usage_MB = 0

# Loop through each sample size
for size in sample_sizes:
    try:
        X_train_sample, X_test_sample, y_train_sample, y_test_sample = sample_data(X, y, size)
        metrics = calculate_metrics(X_train_sample, X_test_sample, y_train_sample, y_test_sample)
        metrics['Sample Size'] = size
        metrics_list.append(metrics)

        # Accumulate total       
        total_execution_time += metrics['Execution Time (Raw)']
        total_cpu_usage += metrics['Average CPU Usage']
        total_memory_usage_MB += X_train_sample.memory_usage(deep=True).sum() / (1024 ** 2)

        print(f"Metrics for sample size {size}:")
        for key, value in metrics.items():
            print(f"{key}: {value}")
        print("-" * 50)

    except Exception as e:
        print(f"An error occurred for sample size {size}: {e}")


Metrics for sample size 1.0:
RMSE: 0.4412058832389192
MAPE: 0.12162312164557083
R2: 0.08439806861845855
MSE: 0.19466263140463477
nRMSE: 0.14706862774630639
Execution Time (Raw): 2943.1918942928314
Normalized Time (s/MB): 4.8592439060344
Average CPU Usage: 23.700000000000003
Sample Size: 1.0
--------------------------------------------------
Metrics for sample size 0.5:
RMSE: 0.4373170375458698
MAPE: 0.11810773952376741
R2: 0.08210423351693641
MSE: 0.19124619132789572
nRMSE: 0.14577234584862328
Execution Time (Raw): 1273.6829977035522
Normalized Time (s/MB): 4.201525262856788
Average CPU Usage: 21.6
Sample Size: 0.5
--------------------------------------------------
Metrics for sample size 0.25:
RMSE: 0.43702107321719147
MAPE: 0.11777181848370433
R2: 0.08334622709803119
MSE: 0.19098741843590583
nRMSE: 0.14567369107239717
Execution Time (Raw): 277.0464870929718
Normalized Time (s/MB): 1.8277983086930891
Average CPU Usage: 39.1
Sample Size: 0.25
-------------------------------------------

In [8]:
# Convert metrics to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Calculate total metrics
total_avg_cpu_usage = total_cpu_usage / len(sample_sizes)
normalized_total_time = total_execution_time / total_memory_usage_MB

# Convert total execution time to minutes and seconds
total_minutes = int(total_execution_time // 60)
total_seconds = total_execution_time % 60

# Display total metrics
print(f"Total Execution Time for Entire Process (Raw): {total_minutes} minutes and {total_seconds:.2f} seconds")
print(f"Total Normalized Execution Time for Entire Process: {normalized_total_time:.8f} seconds per MB")
print(f"Total Average CPU Usage for Entire Process: {total_avg_cpu_usage:.2f}%")

# Display the metrics DataFrame
metrics_df


Total Execution Time for Entire Process (Raw): 77 minutes and 2.49 seconds
Total Normalized Execution Time for Entire Process: 4.03467260 seconds per MB
Total Average CPU Usage for Entire Process: 28.52%


,RMSE,MAPE,R2,MSE,nRMSE,Execution Time (Raw),Normalized Time (s/MB),Average CPU Usage,Sample Size
0,0.441206,0.121623,0.084398,0.194663,0.147069,2943.191894,4.859244,23.70,1.000
1,0.437317,0.118108,0.082104,0.191246,0.145772,1273.682998,4.201525,21.60,0.500
2,0.437021,0.117772,0.083346,0.190987,0.145674,277.046487,1.827798,39.10,0.250
3,0.437503,0.118104,0.081325,0.191409,0.145834,116.955959,1.543220,38.25,0.125
4,0.467651,0.120977,0.158857,0.218697,0.155884,0.470338,55.043025,27.00,100.000
5,0.460004,0.108931,0.050998,0.211604,0.153335,1.544527,18.075374,14.30,1000.000
6,0.442435,0.128351,0.087068,0.195749,0.147478,1.397574,1.635561,32.65,10000.000
7,0.434175,0.118808,0.078841,0.188508,0.144725,8.199389,0.959563,31.55,100000.000


Another test

In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import numpy as np
import time
import psutil
import pandas as pd

In [2]:
import pandas as pd

# Load the dataset with only the necessary columns
selected_columns = ['Severity', 'Description', 'Street', 'City', 'State', 
                    'Country', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)',
                    'Pressure(in)', 'Visibility(mi)', 'Wind_Direction', 
                    'Wind_Speed(mph)', 'Weather_Condition']

df = pd.read_csv('E:/Master_thesis_REV/Datasets/Dataset_US_accidents/archive/US_Accidents_March23.csv', usecols=selected_columns)

# Display the first few rows to check the data
print(df.head())


   Severity                                        Description  \
0         3  Right lane blocked due to accident on I-70 Eas...   
1         2  Accident on Brice Rd at Tussing Rd. Expect del...   
2         2  Accident on OH-32 State Route 32 Westbound at ...   
3         3  Accident on I-75 Southbound at Exits 52 52B US...   
4         2  Accident on McEwen Rd at OH-725 Miamisburg Cen...   

                      Street          City State Country  Temperature(F)  \
0                     I-70 E        Dayton    OH      US            36.9   
1                   Brice Rd  Reynoldsburg    OH      US            37.9   
2             State Route 32  Williamsburg    OH      US            36.0   
3                     I-75 S        Dayton    OH      US            35.1   
4  Miamisburg Centerville Rd        Dayton    OH      US            36.0   

   Wind_Chill(F)  Humidity(%)  Pressure(in)  Visibility(mi) Wind_Direction  \
0            NaN         91.0         29.68            10.0         

In [3]:
# Handle missing values - example with 'Wind_Speed(mph)'
df['Wind_Speed(mph)'].fillna(df['Wind_Speed(mph)'].mean(), inplace=True)

# Frequency encoding for categorical columns
categorical_columns = ['Description', 'Street', 'City', 'State', 'Country', 'Weather_Condition', 'Wind_Direction']

for col in categorical_columns:
    freq = df[col].value_counts()
    df[col] = df[col].map(freq)

# Drop any remaining rows with missing values if necessary
df = df.dropna()

# Display the first few rows after preprocessing
print(df.head())


   Severity  Description   Street     City   State  Country  Temperature(F)  \
2         2          1.0    252.0   3010.0  118115  7728394            36.0   
3         3          5.0  27546.0  24572.0  118115  7728394            35.1   
4         2          2.0    729.0  24572.0  118115  7728394            36.0   
5         3          1.0    104.0   1073.0  118115  7728394            37.9   
6         2          1.0     26.0  24572.0  118115  7728394            34.0   

   Wind_Chill(F)  Humidity(%)  Pressure(in)  Visibility(mi)  Wind_Direction  \
2           33.3        100.0         29.67            10.0        364470.0   
3           31.0         96.0         29.64             9.0        364470.0   
4           33.3         89.0         29.65             6.0        364470.0   
5           35.5         97.0         29.63             7.0        384840.0   
6           31.0        100.0         29.66             7.0        353806.0   

   Wind_Speed(mph)  Weather_Condition  
2         

In [4]:
# Define features (X) and target (y)
X = df.drop(columns=['Severity'])
y = df['Severity']

# Display the shapes of X and y to confirm
print(f'X shape: {X.shape}, y shape: {y.shape}')


X shape: (5676313, 13), y shape: (5676313,)


In [7]:
from sklearn.model_selection import train_test_split

# Function to sample data
def sample_data(X, y, sample_size):
    if isinstance(sample_size, float):
        if 0 < sample_size < 1.0:
            return train_test_split(X, y, test_size=0.2, train_size=sample_size, random_state=42)
        elif sample_size == 1.0:
            return train_test_split(X, y, test_size=0.001, random_state=42)
        else:
            raise ValueError("sample_size as float must be in the range (0.0, 1.0) or equal to 1.0.")
    elif isinstance(sample_size, int):
        if sample_size > len(X):
            raise ValueError(f"sample_size {sample_size} exceeds the number of available samples {len(X)}.")
        sampled_X = X.sample(n=sample_size, random_state=42)
        sampled_y = y.loc[sampled_X.index]
        return train_test_split(sampled_X, sampled_y, test_size=0.2, random_state=42)
    else:
        raise ValueError("sample_size must be a float or an integer")


In [8]:
# Function to calculate and return metrics
def calculate_metrics(X_train, X_test, y_train, y_test):
    rf = RandomForestRegressor(random_state=42)

    # Define hyperparameters for RandomizedSearchCV
    param_distributions = {
        'n_estimators': [50, 100],  # Number of trees
        'max_depth': [2,5, 10],  # Depth of trees
        'min_samples_split': [2, 4, 8, 10],  # Min samples to split
        'min_samples_leaf': [2, 4, 10],  # Min samples in leaf
        'max_features': ['sqrt', 'log2'],  # Features to consider for best split
        'bootstrap': [True]  # Whether to bootstrap samples
    }

    random_search = RandomizedSearchCV(rf, param_distributions, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1)

    start_time = time.time()
    start_cpu = psutil.cpu_percent(interval=None)
    random_search.fit(X_train, y_train)
    end_cpu = psutil.cpu_percent(interval=None)
    end_time = time.time()

    # Calculate time and CPU usage
    execution_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    y_pred = random_search.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    # Calculate the range of the target variable
    target_range = y_train.max() - y_train.min()

    # Calculate normalized RMSE (nRMSE)
    nrmse = rmse / target_range
    
    memory_usage_MB = X_train.memory_usage(deep=True).sum() / (1024 ** 2)
    normalized_time = execution_time / memory_usage_MB
    
    return {
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2,
        'MSE': mse,
        'nRMSE': nrmse,  # Normalized RMSE
        'Execution Time (Raw)': execution_time,  # Raw execution time
        'Normalized Time (s/MB)': normalized_time,  # Normalized execution time
        'Average CPU Usage': avg_cpu_usage
    }


In [9]:
# Define sample sizes
sample_sizes = [1.0, 0.5, 0.25, 0.125, 100, 1000, 10000, 100000]

# Initialize metrics storage
metrics_list = []
total_execution_time = 0
total_cpu_usage = 0
total_memory_usage_MB = 0

# Loop through each sample size
for size in sample_sizes:
    try:
        X_train_sample, X_test_sample, y_train_sample, y_test_sample = sample_data(X, y, size)
        metrics = calculate_metrics(X_train_sample, X_test_sample, y_train_sample, y_test_sample)
        metrics['Sample Size'] = size
        metrics_list.append(metrics)

        # Accumulate total       
        total_execution_time += metrics['Execution Time (Raw)']
        total_cpu_usage += metrics['Average CPU Usage']
        total_memory_usage_MB += X_train_sample.memory_usage(deep=True).sum() / (1024 ** 2)

        print(f"Metrics for sample size {size}:")
        for key, value in metrics.items():
            print(f"{key}: {value}")
        print("-" * 50)

    except Exception as e:
        print(f"An error occurred for sample size {size}: {e}")


Metrics for sample size 1.0:
RMSE: 0.42746299565029816
MAPE: 0.1142781566562957
R2: 0.14054892278827003
MSE: 0.18272461265032683
nRMSE: 0.14248766521676606
Execution Time (Raw): 5920.09191942215
Normalized Time (s/MB): 9.774140326493233
Average CPU Usage: 47.0
Sample Size: 1.0
--------------------------------------------------
Metrics for sample size 0.5:
RMSE: 0.4244556226229372
MAPE: 0.11107089869582622
R2: 0.13530060780885866
MSE: 0.18016257557622525
nRMSE: 0.14148520754097907
Execution Time (Raw): 2670.5573632717133
Normalized Time (s/MB): 8.809424517658398
Average CPU Usage: 44.95
Sample Size: 0.5
--------------------------------------------------
Metrics for sample size 0.25:
RMSE: 0.4246827106046041
MAPE: 0.11110447827488534
R2: 0.13437511474296815
MSE: 0.1803554046864739
nRMSE: 0.14156090353486803
Execution Time (Raw): 1197.3527681827545
Normalized Time (s/MB): 7.899466214343297
Average CPU Usage: 44.75
Sample Size: 0.25
--------------------------------------------------
Metric

In [10]:
# Convert metrics to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Calculate total metrics
total_avg_cpu_usage = total_cpu_usage / len(sample_sizes)
normalized_total_time = total_execution_time / total_memory_usage_MB

# Convert total execution time to minutes and seconds
total_minutes = int(total_execution_time // 60)
total_seconds = total_execution_time % 60

# Display total metrics
print(f"Total Execution Time for Entire Process (Raw): {total_minutes} minutes and {total_seconds:.2f} seconds")
print(f"Total Normalized Execution Time for Entire Process: {normalized_total_time:.8f} seconds per MB")
print(f"Total Average CPU Usage for Entire Process: {total_avg_cpu_usage:.2f}%")

# Display the metrics DataFrame
metrics_df


Total Execution Time for Entire Process (Raw): 172 minutes and 4.21 seconds
Total Normalized Execution Time for Entire Process: 9.01133619 seconds per MB
Total Average CPU Usage for Entire Process: 41.36%


,RMSE,MAPE,R2,MSE,nRMSE,Execution Time (Raw),Normalized Time (s/MB),Average CPU Usage,Sample Size
0,0.427463,0.114278,0.140549,0.182725,0.142488,5920.091919,9.774140,47.00,1.000
1,0.424456,0.111071,0.135301,0.180163,0.141485,2670.557363,8.809425,44.95,0.500
2,0.424683,0.111104,0.134375,0.180355,0.141561,1197.352768,7.899466,44.75,0.250
3,0.424646,0.111050,0.134526,0.180324,0.141549,501.917446,6.622743,46.05,0.125
4,0.465912,0.122488,0.165101,0.217074,0.155304,0.601658,70.411189,36.45,100.000
5,0.453372,0.107680,0.078164,0.205546,0.151124,1.329871,15.563290,31.80,1000.000
6,0.434127,0.122334,0.121031,0.188467,0.144709,3.450109,4.037614,41.65,10000.000
7,0.423991,0.112461,0.121549,0.179768,0.141330,28.908055,3.383068,38.25,100000.000
